https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [2]:
from gym import envs 
# print(envs.registry.all())

all_envs = envs.registry.all()
env_ids = [env_spec.id for env_spec in all_envs]
print("\n".join(env_ids))

CartPole-v0
CartPole-v1
MountainCar-v0
MountainCarContinuous-v0
Pendulum-v1
Acrobot-v1
LunarLander-v2
LunarLanderContinuous-v2
BipedalWalker-v3
BipedalWalkerHardcore-v3
CarRacing-v0
Blackjack-v1
FrozenLake-v1
FrozenLake8x8-v1
CliffWalking-v0
Taxi-v3
Reacher-v2
Pusher-v2
Thrower-v2
Striker-v2
InvertedPendulum-v2
InvertedDoublePendulum-v2
HalfCheetah-v2
HalfCheetah-v3
Hopper-v2
Hopper-v3
Swimmer-v2
Swimmer-v3
Walker2d-v2
Walker2d-v3
Ant-v2
Ant-v3
Humanoid-v2
Humanoid-v3
HumanoidStandup-v2
FetchSlide-v1
FetchPickAndPlace-v1
FetchReach-v1
FetchPush-v1
HandReach-v0
HandManipulateBlockRotateZ-v0
HandManipulateBlockRotateZTouchSensors-v0
HandManipulateBlockRotateZTouchSensors-v1
HandManipulateBlockRotateParallel-v0
HandManipulateBlockRotateParallelTouchSensors-v0
HandManipulateBlockRotateParallelTouchSensors-v1
HandManipulateBlockRotateXYZ-v0
HandManipulateBlockRotateXYZTouchSensors-v0
HandManipulateBlockRotateXYZTouchSensors-v1
HandManipulateBlockFull-v0
HandManipulateBlock-v0
HandManipulate

In [4]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [21]:
import gym
env = gym.make("Hopper-v2").env
env.render()

MujocoDependencyError: To use MuJoCo, you need to either populate ~/.mujoco/mjkey.txt and ~/.mujoco/mjpro131, or set the MUJOCO_PY_MJKEY_PATH and MUJOCO_PY_MJPRO_PATH environment variables appropriately. Follow the instructions on https://github.com/openai/mujoco-py for where to obtain these.

we have six possible actions:

0 = south
1 = north
2 = east
3 = west
4 = pickup
5 = dropoff

Taxi environment has  500 total possible states.
5x5x5x4 = 500 states
5x5 locations where car can be
5 locations where passenger can be (4 pickup location + 1 if inside car)
4 dropoff location 

In [6]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [6]:
state = env.encode(0, 0, 0, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [10]:
state = env.encode(0, 3, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
# blue = passenger pick-up location, 
# purple current destination.
# R,G,Y,B = 0, 1, 2, 3


env.s = state
env.render()


State: 68
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)


Reward Table: P. It's a states X actions Matrix.
Returns  [(probability, nextstate, reward, done)]

In [14]:
env.P[68]


{0: [(1.0, 168, -1, False)],
 1: [(1.0, 68, -1, False)],
 2: [(1.0, 88, -1, False)],
 3: [(1.0, 48, -1, False)],
 4: [(1.0, 68, -10, False)],
 5: [(1.0, 68, -10, False)]}

reward of 20 for correct dropoff and -10 for wrong

## step to execute one action 

In [15]:
env.s = 328  # set environment to illustration's state
state, reward, done, info = env.step(4)
print("state: {}".format(state))
print("reward: {}".format(reward))
print("done: {}".format(done))
print("info: {}".format(info))


    

state: 328
reward: -10
done: False
info: {'prob': 1.0}


## Random Action: env.action_space.sample()

In [16]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 289
Penalties incurred: 95


## Animation


In [17]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 289
State: 0
Action: 5
Reward: 20


# Q-learning

## Q-table

In [18]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [19]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 1s, sys: 15 s, total: 1min 16s
Wall time: 41min 19s


In [20]:
q_table[328]


array([ -2.40804453,  -2.27325184,  -2.4104348 ,  -2.35963784,
       -10.21361783, -10.21419438])

##  Evaluating the agent


In [21]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.24
Average penalties per episode: 0.0


## | Animation

In [22]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

frames = [] # for animation


for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1
            
        # Put each rendered frame into dict for animation
        frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        })

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")





Results after 100 episodes:
Average timesteps per episode: 13.1
Average penalties per episode: 0.0


In [23]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1310
State: 475
Action: 5
Reward: 20
